# CNN Fine Tuning for Cats-Dogs Classification
## CIML Summer Institute, UC San Diego
Fine-tune VGG16 top layer (Conv block 5) and fully connected layers to classify cats vs. dogs. 
Adapted from tensorflow tutorials (https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb)

------


### Setup

In [ ]:
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

from sklearn.metrics import classification_report 
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import time

In [ ]:
print (tf.__version__)
!python --version
print(tf.config.list_physical_devices('GPU'))
!nvidia-smi

In [ ]:
# Set logging level
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# Set random generator seed
seed = 1234

# Disable hash randomization by specifying the value 0.
os.environ['PYTHONHASHSEED'] = '0'

# Set numpy random generator
np.random.seed(seed)

# Set python built-in random generator
random.seed(seed)

# Set tf global random seed
tf.random.set_seed(seed)

# Set tensorflow graph-level random seed
tf.compat.v1.random.set_random_seed(seed)

# Potential randomness from CUDNN
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC']= '1'

### Set image dimensions, location of images

In [ ]:
# Image dimensions
img_width, img_height = 150, 150
IMG_SIZE = (img_width,img_height)
IMG_SHAPE = IMG_SIZE + (3,)

# Location of images
HOME = os.path.expanduser('~') 
data_path = HOME + '/data/catsVsDogs'

train_data_dir = data_path + '/train'
validation_data_dir = data_path + '/val'
test_data_dir = data_path + '/test'

print ('Train path:' + train_data_dir)
print ('Validation:' + validation_data_dir)
print ('Test path:' + test_data_dir)

# Batch size
BATCH_SIZE = 16

### Prepare data

In [ ]:
# Data setup
train_datagen = ImageDataGenerator(rescale = 1. / 255, shear_range = 0.2,
                                   zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1. / 255)
validation_datagen = ImageDataGenerator(rescale = 1. / 255)

# Set up generator to read images found in subfolders of training data directory,
# and indefinitely generate batches of image data (scaled).  This is for training data.
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=IMG_SIZE,
                                                    batch_size = BATCH_SIZE,
                                                    class_mode='binary', 
                                                    shuffle = True,
                                                    seed = seed)           

# Set up generator to generate batched of validation data for model
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size=IMG_SIZE,
                                                              batch_size = BATCH_SIZE,
                                                              class_mode='binary',
                                                              shuffle = False, 
                                                              seed = seed)
# Set up generator to generate batched of test data for model
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=IMG_SIZE,
                                                  batch_size = BATCH_SIZE,
                                                  class_mode='binary',
                                                  shuffle = False,
                                                  seed = seed)

### Load model from feature extraction
Load model saved from feature extraction.

Weights in last convoluational layer and top model will be adjusted.  All other weights are frozen.

In [ ]:
model = tf.keras.models.load_model('weights_from_feature_extract')

In [ ]:
# Freeze all weights of VGG16 model except for conv block5
for layers in model.layers[0].layers[:15]:
    layers.trainable = False
model.summary()

### Fine tune model

In [ ]:
EPOCHS = 20

# Compile model with Adam optimizer with very slow learning rate,
# Binary Cross-Entropy loss function and Accuracy metric
model.compile(optimizer=optimizers.Adam(learning_rate=0.000005),
              loss= losses.BinaryCrossentropy(),metrics=['accuracy'])

# Early Stopping to avoid overfitting and ModelCheckpoint to save the best model
checkpoint_path = 'tmp/checkpoint'
callbacks = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001,
                           mode='min'),
             ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                             mode = 'min', save_best_only = True, 
                             save_weights_only=True)]

In [ ]:
%%time

train_history = model.fit(train_generator,epochs=EPOCHS, 
                          validation_data=validation_generator, 
                          callbacks=callbacks)

In [ ]:
# Load the best model that was saved using ModelCheckpoint
model.load_weights(checkpoint_path)

In [ ]:
# Plot train and validation history
fig, axs = plt.subplots(1,2, figsize= (20,5))
axs[0].plot(train_history.history['loss'])
axs[0].plot(train_history.history['val_loss'])
axs[0].set_title("Train, Val loss history")
axs[0].set_xlabel("Epoch")
axs[0].legend(["Train Loss","Val Loss"])

axs[1].plot(train_history.history['accuracy'])
axs[1].plot(train_history.history['val_accuracy'])
axs[1].set_title("Train, Val Accuracy history")
axs[1].set_xlabel("Epoch")
axs[1].legend(["Train Accuracy","Val Accuracy"])

### Evaluate model

In [ ]:
_, train_accuracy = model.evaluate(train_generator)
print("Train data accuracy:", train_accuracy)

_, test_accuracy = model.evaluate(test_generator)
print("Test data accuracy:", test_accuracy)

In [ ]:
# Get predicted value and the ground truth value of test data
pred = (model.predict(test_generator) > 0.5).astype("int32")
true = test_generator.classes

In [ ]:
# Classification report
print(classification_report(y_true= true, y_pred = pred, target_names=['cats', 'dogs']))

### Perform inference on test images

In [ ]:
def image_loader(img_file):
    """load individual images"""
    img = load_img(img_file, target_size = (img_width, img_height))
    imgplot = plt.imshow(img)
    plt.show()
    img = img_to_array(img) / 255
    img = np.expand_dims(img, axis = 0) #model input is (1,150,150,3)
    return img

In [ ]:
test_image = data_path + '/test/cats/cat.1070.jpg'
img = image_loader(test_image)
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
test_image = data_path + '/test/dogs/dog.1233.jpg'
img = image_loader(test_image)
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
test_image = data_path + '/test/cats/cat.1195.jpg'
img = image_loader(test_image)
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
test_image = data_path + '/test/cats/cat.1011.jpg'
img = image_loader(test_image)
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
test_image = data_path + '/test/dogs/dog.1308.jpg'
img = image_loader(test_image)
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
test_image = data_path + '/test/dogs/dog.1342.jpg'
img = image_loader(test_image)
img_y_pred = model.predict(img) 
print(img_y_pred)